# Multi-task Sequential Federated Learning on MedMNIST

In [ ]:
# ! pip install medmnist

In [1]:
# import EarlyStopping
from utils.pytorchtools import EarlyStopping

In [2]:
# Import libraries
import medmnist
from medmnist import INFO

from tqdm import tqdm
import time
import numpy as np
from statistics import mean
import PIL
from itertools import chain
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support

import torch
import torch.nn as nn
import torchvision.transforms as transforms

# Models
from models.CNN import CNN5
from models.VGG11 import VGG11
from models.ResNet18 import ResNet_18, Block

# Train and Test/validation
from Training import Train
from Evaluation import Evaluator

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [3]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

Using device: cpu



## Preparing Dataset including Server and Clients

In [4]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v2.2.3 @ https://github.com/MedMNIST/MedMNIST/


In [5]:
# getting all data flags in MedMNIST
medmnist.INFO

{'pathmnist': {'python_class': 'PathMNIST',
  'description': 'The PathMNIST is based on a prior study for predicting survival from colorectal cancer histology slides, providing a dataset (NCT-CRC-HE-100K) of 100,000 non-overlapping image patches from hematoxylin & eosin stained histological images, and a test dataset (CRC-VAL-HE-7K) of 7,180 image patches from a different clinical center. The dataset is comprised of 9 types of tissues, resulting in a multi-class classification task. We resize the source images of 3×224×224 into 3×28×28, and split NCT-CRC-HE-100K into training and validation set with a ratio of 9:1. The CRC-VAL-HE-7K is treated as the test set.',
  'url': 'https://zenodo.org/record/6496656/files/pathmnist.npz?download=1',
  'MD5': 'a8b06965200029087d5bd730944a56c1',
  'task': 'multi-class',
  'label': {'0': 'adipose',
   '1': 'background',
   '2': 'debris',
   '3': 'lymphocytes',
   '4': 'mucus',
   '5': 'smooth muscle',
   '6': 'normal colon mucosa',
   '7': 'cancer-as

In [6]:
# List of datasets that we are going to use in this experiment
data_flags = ['octmnist', 'organamnist', 'tissuemnist']
num_tasks = 3

In [7]:
def get_transform():
  '''
  preprocessing
  '''
  data_transform = transforms.Compose([
      transforms.Resize((224, 224), interpolation=PIL.Image.NEAREST),
      transforms.ToTensor(),
      transforms.Normalize(mean=[.5], std=[.5])
  ])

  return data_transform

In [8]:
def get_dataset(dataset_name='pathmnist', split='train'):
  '''
  load the data
  '''
  info = INFO[dataset_name]
  DataClass = getattr(medmnist, info['python_class']) # To load selected MedMNIST dataset

  data_transform = get_transform()
  dataset = DataClass(split=split, transform=data_transform, download=True)

  return dataset, info

In [9]:
dataflag_to_dataset = dict()

for data_flag in data_flags:
  dataset, info = get_dataset(dataset_name = data_flag, split='train')
  dataflag_to_dataset[data_flag] = (dataset, info)

Using downloaded and verified file: /Users/atefe/.medmnist/octmnist.npz


Using downloaded and verified file: /Users/atefe/.medmnist/organamnist.npz
Using downloaded and verified file: /Users/atefe/.medmnist/tissuemnist.npz


In [10]:
info = INFO['octmnist']
n_channels = info['n_channels']
print("n_channels:", n_channels)

n_channels: 1


In [11]:
# Splitting server into train/test dataset while shifting the labels for non-federated 
# For example: labels 0-3 -> 0-3, 0-4 -> 4-8

amount = 8000
shift = 0
split_ratio = 0.2
server_train_data, server_train_labels, server_test_data, server_test_labels = list(), list(), list(), list()

for data_flag, (data, info) in dataflag_to_dataset.items():
  print(data_flag)
  print(len(data.imgs))
  dataset = data.imgs[0: amount]
  print(len(dataset))
  # select the amount of labels and make it falt
  labels = list(np.ravel(data.labels[0: amount]))

  labels = [label + shift for label in labels]
  shift += len(dataflag_to_dataset[data_flag][1]['label'].keys())

  server_train_data.append(dataset[int(amount * split_ratio):])
  server_train_labels.append(labels[int(amount * split_ratio):])

  server_test_data.append(dataset[0: int(amount * split_ratio)])
  server_test_labels.append(labels[0: int(amount * split_ratio)])

server_train_data_centralized = np.concatenate(server_train_data, axis=0)
server_train_labels_centralized = np.concatenate(server_train_labels, axis=0)

server_test_data_centralized = np.concatenate(server_test_data, axis=0)
server_test_labels_centralized = np.concatenate(server_test_labels, axis=0)

print("server_train_data_centralized shape:", server_train_data_centralized.shape)
print("server_train_labels_centralized shape:", server_train_labels_centralized.shape)
print("server_test_data_centralized shape:", server_test_data_centralized.shape)
print("server_test_labels_centralized shape:", server_test_labels_centralized.shape)

octmnist
97477
8000
organamnist
34581
8000
tissuemnist
165466
8000
server_train_data_centralized shape: (19200, 28, 28)
server_train_labels_centralized shape: (19200,)
server_test_data_centralized shape: (4800, 28, 28)
server_test_labels_centralized shape: (4800,)


In [12]:
# Shuffeling the server data and labels simultaneously
idx_train = np.random.permutation(len(server_train_labels_centralized))
server_train_data_centralized, server_train_labels_centralized = server_train_data_centralized[idx_train], server_train_labels_centralized[idx_train]

In [13]:
server_valid_data_centralized = server_train_data_centralized[0:int(0.1 * len(server_train_data_centralized))]
server_valid_labels_centralized = server_train_labels_centralized[0:int(0.1 * len(server_train_labels_centralized))]

server_train_data_centralized = server_train_data_centralized[int(0.1 * len(server_train_data_centralized)):]
server_train_labels_centralized = server_train_labels_centralized[int(0.1 * len(server_train_labels_centralized)):]

In [14]:
# Shuffeling the server data and labels simultaneously
idx_test = np.random.permutation(len(server_test_data_centralized))
server_test_data_centralized, server_test_labels_centralized = server_test_data_centralized[idx_test], server_test_labels_centralized[idx_test]

In [15]:
server_train_data_centralized = server_train_data_centralized.astype('float32')
server_test_data_centralized = server_test_data_centralized.astype('float32')
server_valid_data_centralized = server_valid_data_centralized.astype('float32')

server_train_data_centralized = server_train_data_centralized[:,None,:,:]
server_train_labels_centralized = server_train_labels_centralized[:, None]

server_valid_data_centralized = server_valid_data_centralized[:,None,:,:]
server_valid_labels_centralized = server_valid_labels_centralized[:, None]

server_test_data_centralized = server_test_data_centralized[:,None,:,:]
server_test_labels_centralized = server_test_labels_centralized[:, None]

server_train_data_centralized, server_train_labels_centralized, server_valid_data_centralized, server_valid_labels_centralized, server_test_data_centralized, server_test_labels_centralized  = map(torch.tensor, (server_train_data_centralized, server_train_labels_centralized, server_valid_data_centralized, server_valid_labels_centralized, server_test_data_centralized, server_test_labels_centralized ))


In [16]:
# Define a custom transformation function to resize the images
def resize_transform(image_tensor, new_height, new_width):
  image_pil = transforms.ToPILImage()(image_tensor)
  resized_image = transforms.Resize((new_height, new_width))(image_pil)
  return transforms.ToTensor()(resized_image)

In [17]:
server_batch_size = 64

# Apply the custom transformation to the entire dataset
resized_images_train = [resize_transform(image, 32, 32) for image in server_train_data_centralized]
train_ds_server = TensorDataset(torch.stack(resized_images_train), server_train_labels_centralized)

resized_images_valid= [resize_transform(image, 32, 32) for image in server_valid_data_centralized]
valid_ds_server = TensorDataset(torch.stack(resized_images_valid), server_valid_labels_centralized)

resized_images_test = [resize_transform(image, 32, 32) for image in server_test_data_centralized]
test_ds_server = TensorDataset(torch.stack(resized_images_test), server_test_labels_centralized)
###

train_dl_server = DataLoader(train_ds_server, batch_size=server_batch_size, shuffle=True)
valid_dl_server = DataLoader(valid_ds_server, batch_size=server_batch_size, shuffle=True)
test_dl_server = DataLoader(test_ds_server, batch_size=server_batch_size)

decentralized  data

## Classification Model

In [18]:
def get_model(model_name):
  if model_name == 'CNN5':
    return CNN5()
  if model_name == 'VGG11':
    return VGG11()
  elif model_name == 'ResNet_18':
    return ResNet_18()

In [19]:
# def train(model, train_loader, criterion, optimizer):

#   model.train()
#   train_loss = 0.0
#   correct = 0
#   conf_matrix = torch.zeros(n_classes, n_classes)
#   # metrics variables
#   accuracy, precision, recall, f1 = [], [], [], []

#   for data, target in train_loader:

#     optimizer.zero_grad()
#     output = model(data)

#     target = target.squeeze().long()
#     loss = criterion(output, target)

#     loss.backward()
#     optimizer.step()

#     train_loss += loss.item()
#     prediction = output.argmax(dim=1, keepdim=True)
#     correct += prediction.eq(target.view_as(prediction)).sum().item()

#     accuracy.append(accuracy_score(prediction, target))

#     # Calculate Sensitivity, Specificity, and F1-score for each class
#     metrics = precision_recall_fscore_support(prediction, target, average='macro', zero_division=0.0)
#     precision_e, recall_e, f1_e, _ = metrics

#     precision.append(precision_e)
#     recall.append(recall_e)
#     f1.append(f1_e)

#     for t, p in zip(target, prediction):
#       conf_matrix[t, p] += 1

#   TP = conf_matrix.diag()
#   specificity = []
#   for c in range(n_classes):
#       idx = torch.ones(n_classes).byte()
#       idx[c] = 0
#       # all non-class samples classified as non-class
#       TN = conf_matrix[idx.nonzero()[:, None], idx.nonzero()].sum() #conf_matrix[idx[:, None], idx].sum() - conf_matrix[idx, c].sum()
#       # all non-class samples classified as class
#       FP = conf_matrix[idx, c].sum()
#       # all class samples not classified as class
#       FN = conf_matrix[c, idx].sum()
    
#       # print('Class {}\nTP {}, TN {}, FP {}, FN {}'.format(c, TP[c], TN, FP, FN))

#       specificity_c = TN / (TN + FP)
#       specificity.append(specificity_c)

#   # Print the results
#   # perfromance metrics per epoch 
#   # print(f'Accuracy: {np.mean(accuracy):.4f}')
#   # print(f'Sensitivity (Recall): {np.mean(recall):.4f}')
#   # print(f'Specificity: {np.mean(specificity):.4f}')
#   # print(f'F1-score: {np.mean(f1):.4f}')
 

#   return train_loss/len(train_loader), correct/len(train_loader.dataset), np.mean(accuracy), np.mean(recall), np.mean(specificity), np.mean(f1)

In [20]:
# def validation(model, test_loader, criterion):
#   model.eval()
#   test_loss = 0
#   correct = 0
#   with torch.no_grad():
#     for data, target in test_loader:
#       output = model(data)
#       target = target.squeeze().long()
#       test_loss += criterion(output, target).item()
#       prediction = output.argmax(dim=1, keepdim=True)
#       correct += prediction.eq(target.view_as(prediction)).sum().item()
#   # test_loss = 0 if len(test_loader) == 0 else test_loss /= len(test_loader)
#   if len(test_loader) == 0:
#     # print("Can't perform division: the y variable is 0")
#     test_loss = 0
#   else:
#     test_loss /= len(test_loader)
#     # test_loss /= len(test_loader)
#   if len(test_loader.dataset) == 0:
#     # print("Can't perform division: the yy variable is 0")
#     correct = 0
#   else:
#     correct /= len(test_loader.dataset)
#     # correct /= len(test_loader.dataset)

#   return (test_loss, correct)

## Added for New Eval Metrics
> WE need to specify the number of classes for each dataset in the evaluation pipeline.

In [21]:
# def validation(model, test_loader, criterion):
#   model.eval()
#   test_loss = 0
#   correct = 0
#   conf_matrix = torch.zeros(n_classes, n_classes)
#   # metrics variables
#   accuracy, precision, recall, f1 = [], [], [], []
  
#   with torch.no_grad():
#     for data, target in test_loader:
#       output = model(data)
#       target = target.squeeze().long()
#       test_loss += criterion(output, target).item()
#       prediction = output.argmax(dim=1, keepdim=True)
#       correct += prediction.eq(target.view_as(prediction)).sum().item()
      
#       # accuracy per batch
#       accuracy.append(accuracy_score(prediction, target))

#       # Calculate Sensitivity, Specificity, and F1-score for each class
#       metrics = precision_recall_fscore_support(prediction, target, average='macro', zero_division=0.0)
#       precision_e, recall_e, f1_e, _ = metrics

#       precision.append(precision_e)
#       recall.append(recall_e)
#       f1.append(f1_e)

#       for t, p in zip(target, prediction):
#         conf_matrix[t, p] += 1

#   TP = conf_matrix.diag()
#   specificity = []
#   for c in range(n_classes):
#       idx = torch.ones(n_classes).byte()
#       idx[c] = 0
#       # all non-class samples classified as non-class
#       TN = conf_matrix[idx.nonzero()[:, None], idx.nonzero()].sum() #conf_matrix[idx[:, None], idx].sum() - conf_matrix[idx, c].sum()
#       # all non-class samples classified as class
#       FP = conf_matrix[idx, c].sum()
#       # all class samples not classified as class
#       FN = conf_matrix[c, idx].sum()
    
#       # print('Class {}\nTP {}, TN {}, FP {}, FN {}'.format(c, TP[c], TN, FP, FN))

#       specificity_c = TN / (TN + FP)
#       specificity.append(specificity_c)

#   # Print the results
#   # perfromance metrics per epoch 
#   # print(f'Accuracy: {np.mean(accuracy):.4f}')
#   # print(f'Sensitivity (Recall): {np.mean(recall):.4f}')
#   # print(f'Specificity: {np.mean(specificity):.4f}')
#   # print(f'F1-score: {np.mean(f1):.4f}')

#   # test_loss = 0 if len(test_loader) == 0 else test_loss /= len(test_loader)
#   if len(test_loader) == 0:
#     # print("Can't perform division: the y variable is 0")
#     test_loss = 0
#   else:
#     test_loss /= len(test_loader)
    
#   if len(test_loader.dataset) == 0:
#     # print("Can't perform division: the yy variable is 0")
#     correct = 0
#   else:
#     correct /= len(test_loader.dataset)

#   return (test_loss, correct, np.mean(accuracy), np.mean(recall), np.mean(specificity), np.mean(f1))

In [22]:
def create_file():
    baseline_metrics_train= open(f"results/{STRATEGY}/{MODEL}/{SPILT}/baseline_train_lr{learning_rate}_bs{batch_size}.txt", 'w')
    baseline_metrics_train.write("iteration\ttrain_accuracy\ttrain_loss\t train_accuracy_skl\ttrain_sensitivity\ttrain_specificity\ttrain_f1\n")
     
    baseline_metrics_validation= open(f"results/{STRATEGY}/{MODEL}/{SPILT}/baseline_validation_lr{learning_rate}_bs{batch_size}.txt", 'w')
    baseline_metrics_validation.write("iteration\tvalid_accuracy\t valid_accuracy_skl\tvalid_loss\tvalid_sensitivity\tvalid_specificity\tvalid_f1\n")
    
    return baseline_metrics_train, baseline_metrics_validation

### Parameter Initialization: Models, Optimizers, and Loss Functions

In [23]:
learning_rate = 0.0001
batch_size = 64
momentum = 0.9
n_classes = 23
n_channels = 1

STRATEGY = "NonFederated"

SPILT = "Random"
# MODEL = ['CNN5', 'VGG11', 'ResNet18']
MODEL = 'CNN5'
NUM_ITERATION = 50


In [24]:
base_model = get_model(model_name=MODEL)
# base_model (to print the architecture of the model)

In [25]:
baseline_metrics_train, baseline_metrics_validation = create_file()

In [26]:
print(f"Model is {MODEL}")

start_server_training_time = time.time()

# train server on mixed data.
main_model = get_model(model_name = MODEL)
main_optimizer = torch.optim.SGD(main_model.parameters(), lr=learning_rate, momentum=0.9)
main_criterion = nn.CrossEntropyLoss()

# early stopping patience; how long to wait after last time validation loss improved.
patience = 5
# initialize the early_stopping object
early_stopping = EarlyStopping(patience=patience, verbose=True)

trainer = Train(n_classes)
validator = Evaluator(n_classes)

for iter in range(NUM_ITERATION):
  print("-------Baseline result----------")
  # model training
  central_train_loss, central_train_accuracy, central_train_accuracy_skl, central_train_sens, central_train_spec, central_train_f1 = trainer.train(main_model, train_dl_server, main_criterion, main_optimizer)
  # model validaiton
  central_valid_loss, central_valid_accuracy, central_valid_accuracy_skl, central_valid_sens, central_valid_spec, central_valid_f1 = validator.validation(main_model, valid_dl_server, main_criterion)
  
  print("epoch: {:3.0f}".format(iter+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) +  " | valid accuracy: {:7.4f}".format(central_valid_accuracy))
  print("epoch: {:3.0f}".format(iter+1) + " | train loss: {:7.4f}".format(central_train_loss) + " | valid loss: {:7.4f}".format(central_valid_loss))

  # early_stopping needs the validation loss to check if it has decresed, 
  # and if it has, it will make a checkpoint of the current model
  early_stopping(central_valid_loss, main_model)
        
  if early_stopping.early_stop:
    print("Early stopping")
    break

  
  # ("iteration\ttrain_accuracy\ttrain_loss\tvalid_accuracy\t valid_accuracy_skl\tvalid_loss\tvalid_sensitivity\tvalid_specificity\tvalid_f1\n")
  baseline_metrics_train.write(f"{iter}\t{central_train_accuracy}\t{central_train_loss}\t{central_train_accuracy_skl}\t{central_train_sens}\t{central_train_spec}\t{central_train_f1}\n")
  baseline_metrics_validation.write(f"{iter}\t{central_valid_accuracy}\t{central_valid_accuracy_skl}\t{central_valid_loss}\t{central_valid_sens}\t{central_valid_spec}\t{central_valid_f1}\n")

# Testing final model
central_test_loss, central_test_accuracy, central_test_accuracy_skl, central_test_sens, central_test_spec, central_test_f1 = validator.validation(main_model, test_dl_server, main_criterion)

print("final test accuracy: {:7.4f}".format(central_test_accuracy))

end_server_training_time = time.time()
elapsed_server_training_time = end_server_training_time - start_server_training_time
print("elapsed_server_training_time: ", elapsed_server_training_time)

baseline_metrics_train.close()
baseline_metrics_validation.close()

Model is CNN5
-------Baseline result----------


/Users/atefe/Desktop/TestClass/Training.py:55: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FP = conf_matrix[idx, c].sum()
/Users/atefe/Desktop/TestClass/Training.py:57: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FN = conf_matrix[c, idx].sum()
/Users/atefe/Desktop/TestClass/Evaluation.py:47: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FP = conf_matrix[idx, c].sum()
/Users/atefe/Desktop/TestClass/Evaluation.py:49: UserWarning: indexing with dtype torch.uint8 is now deprecated, please

epoch:   1 | train accuracy:  0.0987 | valid accuracy:  0.2333
epoch:   1 | train loss:  3.0986 | valid loss:  3.0183
Validation loss decreased (inf --> 3.018330).  Saving model ...
-------Baseline result----------


/Users/atefe/Desktop/TestClass/Training.py:55: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FP = conf_matrix[idx, c].sum()
/Users/atefe/Desktop/TestClass/Training.py:57: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FN = conf_matrix[c, idx].sum()
/Users/atefe/Desktop/TestClass/Evaluation.py:47: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FP = conf_matrix[idx, c].sum()
/Users/atefe/Desktop/TestClass/Evaluation.py:49: UserWarning: indexing with dtype torch.uint8 is now deprecated, please

epoch:   2 | train accuracy:  0.2163 | valid accuracy:  0.2510
epoch:   2 | train loss:  2.9287 | valid loss:  2.7487
Validation loss decreased (3.018330 --> 2.748716).  Saving model ...
-------Baseline result----------


...

/Users/atefe/Desktop/TestClass/Training.py:55: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FP = conf_matrix[idx, c].sum()
/Users/atefe/Desktop/TestClass/Training.py:57: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FN = conf_matrix[c, idx].sum()
/Users/atefe/Desktop/TestClass/Evaluation.py:47: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FP = conf_matrix[idx, c].sum()
/Users/atefe/Desktop/TestClass/Evaluation.py:49: UserWarning: indexing with dtype torch.uint8 is now deprecated, please

epoch:  49 | train accuracy:  0.6470 | valid accuracy:  0.6703
epoch:  49 | train loss:  0.9952 | valid loss:  0.9263
EarlyStopping counter: 3 out of 5
-------Baseline result----------


/Users/atefe/Desktop/TestClass/Training.py:55: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FP = conf_matrix[idx, c].sum()
/Users/atefe/Desktop/TestClass/Training.py:57: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FN = conf_matrix[c, idx].sum()
/Users/atefe/Desktop/TestClass/Evaluation.py:47: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FP = conf_matrix[idx, c].sum()
/Users/atefe/Desktop/TestClass/Evaluation.py:49: UserWarning: indexing with dtype torch.uint8 is now deprecated, please

epoch:  50 | train accuracy:  0.6464 | valid accuracy:  0.6760
epoch:  50 | train loss:  0.9917 | valid loss:  0.9129
EarlyStopping counter: 4 out of 5
final test accuracy:  0.6679
elapsed_server_training_time:  1101.3369817733765


/Users/atefe/Desktop/TestClass/Evaluation.py:47: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FP = conf_matrix[idx, c].sum()
/Users/atefe/Desktop/TestClass/Evaluation.py:49: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  FN = conf_matrix[c, idx].sum()
